In [9]:
import numpy as np
import torch
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from modules import NetModule
from backdoor import Backdoor

In [10]:
backdoor = Backdoor(ConvNet, config='11-layer', channels=3, classes=10)
backdoor.load_data(torchvision.datasets.CIFAR10, etype=ImageEntity)
backdoor.create_models(optim.SGD, torch.nn.CrossEntropyLoss, lr=0.01)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
patches = ImagePatch((9, 9), 3, 'random').get_patches(len(backdoor))
targets = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 0}
operations = (None, [RotateTransform()], [LabelTransform(targets)])
merge = OverlayMerge(select=True)

In [12]:
backdoor.poison(patches, operations, merge, 0.2)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
backdoor.train(2, epochs=1, verbose=True, device=device)

Training started
Epoch 1/1 | 24.95% | Loss: 1.7645 | Samples trained: 12480/50000
Epoch 1/1 | 49.90% | Loss: 1.5738 | Samples trained: 24960/50000
Epoch 1/1 | 74.86% | Loss: 1.4508 | Samples trained: 37440/50000
Epoch 1/1 | 99.81% | Loss: 1.3611 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.3608
Training complete | Net Average Loss: 1.3608 | Total epochs: 1
Training started
Epoch 1/1 | 24.95% | Loss: 1.9560 | Samples trained: 12480/50000
Epoch 1/1 | 49.90% | Loss: 1.7885 | Samples trained: 24960/50000
Epoch 1/1 | 74.86% | Loss: 1.6654 | Samples trained: 37440/50000
Epoch 1/1 | 99.81% | Loss: 1.5692 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.5689
Training complete | Net Average Loss: 1.5689 | Total epochs: 1


(1.3607598272028902, 1.5688932936922222)

In [15]:
backdoor.eval(device=device)

Accuracy on Clean | Base 56.77% | Trojan 49.14% | Difference 7.63%
Base Accuracy on Poison 2.91% | Attack Success Rate (ASR): 27.70%
Average Tensor Distance: 302.80 | Net Difference 15139797.77


(0.5677, 0.4914, 0.0291, 0.3061, 302.795955300293, 15139797.765014648)